## Cargo la base de tiendas que tengo que analizar

In [ ]:
import pandas as pd
from datetime import datetime

In [288]:
df = pd.read_csv('data/test/venta por media hora.csv', encoding='utf-16', header=2)

C:\Users\juan_mera\AppData\Local\Temp\ipykernel_21164\2695994929.py:1: DtypeWarning: Columns (0,4,13,106,458,459,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/test/venta por media hora.csv', encoding='utf-16', header=2)


In [289]:
df = df.copy()

In [290]:
df = df[1:]

In [291]:
df = df.rename(columns={
    ' ':'año',
    ' .2':'fecha',
    ' .3':'hora'
})

In [292]:
df = df.drop(columns=[' .1', 'Punto Operacional'])

In [293]:
df = df.melt(id_vars=['año', 'fecha', 'hora'], var_name='tiendas', value_name='vst').dropna(subset=['vst'])

In [294]:
df = df.reset_index().drop(columns=['index'])

In [295]:
df['vst'] = pd.to_numeric(df['vst'].str.replace('.', '').str.replace(',', '.'), errors='coerce')

In [296]:
df['mes'] = df['fecha'].str.strip().str.split(' ').str[2].str.strip()

In [297]:
df['dia'] = df['fecha'].str.strip().str.split(' ').str[0].astype(str)

In [298]:
df['no'] = df['tiendas'].str.split(' ').str[0].astype(int)

In [299]:
mes = 'Enero Febrero Marzo Abril Mayo Junio Julio Agosto Septiembre Octubre Noviembre Diciembre'.split(' ')
num_mes = '1 2 3 4 5 6 7 8 9 10 11 12'.split(' ')
mes_orden = dict(zip(mes, num_mes))

In [300]:
df['mes_numerico'] = df['mes'].map(mes_orden)

In [301]:
df['fecha_parsed'] = df['mes_numerico'] + '/' + df['dia'] + '/' + df['año'].astype(str)

In [303]:
df['fecha_final'] = pd.to_datetime(df['fecha_parsed'], format='%m/%d/%Y')

In [304]:
df['nombre_dia'] = df['fecha_final'].dt.day_name()

In [305]:
df_domingos = df[df['nombre_dia'] == 'Sunday']

## Trabajo sobre la muestra de tiendas a analizar

In [307]:
pad = pd.read_excel('data/test/padron_horario_extendido.xlsx')

In [308]:
pad.columns = pad.columns.str.lower().str.strip()

In [309]:
pad

,no,nombre,localidad,zona,gerente regional,domingos,feriados,horario ant,grupo
0,200,Avenida Rivadavia 8421,CABA,4,Pablo Chaperon,8 a 22,8 a 22,11:00-20:00,G11
1,213,Paraguay 919,CABA,13,Debora Pereyra,8 a 22,8 a 22,11:00-20:00,G11
2,251,Arce 730,CABA,8,Matias Flores,8 a 22,8 a 22,8:00-21:00,G12
3,260,Las Heras 2397,CABA,13,Debora Pereyra,8 a 22,8 a 22,11:00-20:00,G11
4,291,Piedras 383,CABA,14,Roman Fleitas,8 a 21:30,8 a 21:30,10:00-19:00,G6
...,...,...,...,...,...,...,...,...,...
92,757,Av. Santa Fe 2429,CABA,13,Debora Pereyra,8 a 22,8 a 22,11:00-20:00,G11
93,773,Pres. Manuel Quintana 570,CABA,13,Debora Pereyra,8 a 22,8 a 22,11:00-20:00,G11
94,787,Alvear 2446,GBA,1,Nestor Flores,8:30 a 21,8:30 a 21,9:00-20:00,G15
95,802,Artigas 4925,GBA,1,Nestor Flores,8 a 21,8 a 21,9:00-20:00,G15


### Cargo el detalle de Horas Autorizadas para las tiendas

In [310]:
horas_autorizadas = pd.read_excel('data/test/padron_horario_extendido.xlsx', sheet_name='horas_dom')

In [311]:
horas_autorizadas.columns = pd.to_datetime(horas_autorizadas.columns, errors='coerce').month_name() if isinstance(horas_autorizadas.columns, datetime) else horas_autorizadas.columns

In [312]:
for col in horas_autorizadas.columns:
    if isinstance(col, datetime):
        horas_autorizadas = horas_autorizadas.rename(columns={
            col:pd.to_datetime(col).month_name().lower().strip()
        })
    else:
        horas_autorizadas = horas_autorizadas.rename(columns={
            col:col.lower().strip()
        })

In [313]:
horas_autorizadas.columns

Index(['no', 'nombre', 'localidad', 'zona', 'gerente regional', 'domingos',
       'april', 'may', 'june', 'july', 'august', 'september'],
      dtype='object')

In [314]:
horas_autorizadas = horas_autorizadas.melt(id_vars=['no', 'nombre', 'localidad', 'zona', 'gerente regional', 'domingos'], value_name='horas_autorizadas', var_name='mes').drop(columns=['domingos']).sort_values(by='no')

In [315]:
horas_autorizadas

,no,nombre,localidad,zona,gerente regional,mes,horas_autorizadas
542,200,Avenida Rivadavia 8421,CABA,4,Pablo Chaperon,september,124.5
443,200,Avenida Rivadavia 8421,CABA,4,Pablo Chaperon,august,118.0
245,200,Avenida Rivadavia 8421,CABA,4,Pablo Chaperon,june,167.0
344,200,Avenida Rivadavia 8421,CABA,4,Pablo Chaperon,july,98.5
47,200,Avenida Rivadavia 8421,CABA,4,Pablo Chaperon,april,76.0
...,...,...,...,...,...,...,...
395,854,Avenida Bartolome Mitre 3125/3145,Gran Buenos Aires,1,Nestor Flores,july,61.5
98,854,Avenida Bartolome Mitre 3125/3145,Gran Buenos Aires,1,Nestor Flores,april,46.0
494,854,Avenida Bartolome Mitre 3125/3145,Gran Buenos Aires,1,Nestor Flores,august,71.0
197,854,Avenida Bartolome Mitre 3125/3145,Gran Buenos Aires,1,Nestor Flores,may,16.5


In [316]:
pad

,no,nombre,localidad,zona,gerente regional,domingos,feriados,horario ant,grupo
0,200,Avenida Rivadavia 8421,CABA,4,Pablo Chaperon,8 a 22,8 a 22,11:00-20:00,G11
1,213,Paraguay 919,CABA,13,Debora Pereyra,8 a 22,8 a 22,11:00-20:00,G11
2,251,Arce 730,CABA,8,Matias Flores,8 a 22,8 a 22,8:00-21:00,G12
3,260,Las Heras 2397,CABA,13,Debora Pereyra,8 a 22,8 a 22,11:00-20:00,G11
4,291,Piedras 383,CABA,14,Roman Fleitas,8 a 21:30,8 a 21:30,10:00-19:00,G6
...,...,...,...,...,...,...,...,...,...
92,757,Av. Santa Fe 2429,CABA,13,Debora Pereyra,8 a 22,8 a 22,11:00-20:00,G11
93,773,Pres. Manuel Quintana 570,CABA,13,Debora Pereyra,8 a 22,8 a 22,11:00-20:00,G11
94,787,Alvear 2446,GBA,1,Nestor Flores,8:30 a 21,8:30 a 21,9:00-20:00,G15
95,802,Artigas 4925,GBA,1,Nestor Flores,8 a 21,8 a 21,9:00-20:00,G15
